## Imports and logging

In [2]:
import os
import re
import json
import jsonschema
import pandas as pd
import numpy as np
import logging
import pytz
import pydicom
import string
import tzlocal
import logging
import zipfile
import datetime
import argparse
import nibabel
from fnmatch import fnmatch
from pprint import pprint

logging.basicConfig()
log = logging.getLogger('U2787g')

## Functions

In [3]:
def get_session_label(dcm):
    """
    Switch on manufacturer and either pull out the StudyID or the StudyInstanceUID
    """
    session_label = ''
    if ( dcm.get('Manufacturer') and (dcm.get('Manufacturer').find('GE') != -1 or dcm.get('Manufacturer').find('Philips') != -1 ) and dcm.get('StudyID')):
        session_label = dcm.get('StudyID')
    else:
        session_label = dcm.get('StudyInstanceUID')

    return session_label


def validate_timezone(zone):
    # pylint: disable=missing-docstring
    if zone is None:
        zone = tzlocal.get_localzone()
    else:
        try:
            zone = pytz.timezone(zone.zone)
        except pytz.UnknownTimeZoneError:
            zone = None
    return zone


def parse_patient_age(age):
    """
    Parse patient age from string.
    convert from 70d, 10w, 2m, 1y to datetime.timedelta object.
    Returns age as duration in seconds.
    """
    if age == 'None' or not age:
        return None

    conversion = {  # conversion to days
        'Y': 365.25,
        'M': 30,
        'W': 7,
        'D': 1,
    }
    scale = age[-1:]
    value = age[:-1]
    if scale not in conversion.keys():
        # Assume years
        scale = 'Y'
        value = age

    age_in_seconds = datetime.timedelta(int(value) * conversion.get(scale)).total_seconds()

    # Make sure that the age is reasonable
    if not age_in_seconds or age_in_seconds <= 0:
        age_in_seconds = None

    return age_in_seconds


def timestamp(date, time, timezone):
    """
    Return datetime formatted string
    """
    if date and time and timezone:
        # return datetime.datetime.strptime(date + time[:6], '%Y%m%d%H%M%S')
        try:
            return timezone.localize(datetime.datetime.strptime(date + time[:6], '%Y%m%d%H%M%S'), timezone)
        except:
            log.warning('Failed to create timestamp!')
            log.info(date)
            log.info(time)
            log.info(timezone)
            return None
    return None


def get_timestamp(dcm, timezone):
    """
    Parse Study Date and Time, return acquisition and session timestamps
    """
    if hasattr(dcm, 'StudyDate') and hasattr(dcm, 'StudyTime'):
        study_date = dcm.StudyDate
        study_time = dcm.StudyTime
    elif hasattr(dcm, 'StudyDateTime'):
        study_date = dcm.StudyDateTime[0:8]
        study_time = dcm.StudyDateTime[8:]
    else:
        study_date = None
        study_time = None

    if hasattr(dcm, 'AcquisitionDate') and hasattr(dcm, 'AcquisitionTime'):
        acquitision_date = dcm.AcquisitionDate
        acquisition_time = dcm.AcquisitionTime
    elif hasattr(dcm, 'AcquisitionDateTime'):
        acquitision_date = dcm.AcquisitionDateTime[0:8]
        acquisition_time = dcm.AcquisitionDateTime[8:]
    # The following allows the timestamps to be set for ScreenSaves
    elif hasattr(dcm, 'ContentDate') and hasattr(dcm, 'ContentTime'):
        acquitision_date = dcm.ContentDate
        acquisition_time = dcm.ContentTime
    else:
        acquitision_date = None
        acquisition_time = None

    session_timestamp = timestamp(dcm.StudyDate, dcm.StudyTime, timezone)
    acquisition_timestamp = timestamp(acquitision_date, acquisition_time, timezone)

    if session_timestamp:
        if session_timestamp.tzinfo is None:
            log.info('no tzinfo found, using UTC...')
            session_timestamp = pytz.timezone('UTC').localize(session_timestamp)
        session_timestamp = session_timestamp.isoformat()
    else:
        session_timestamp = ''
    if acquisition_timestamp:
        if acquisition_timestamp.tzinfo is None:
            log.info('no tzinfo found, using UTC')
            acquisition_timestamp = pytz.timezone('UTC').localize(acquisition_timestamp)
        acquisition_timestamp = acquisition_timestamp.isoformat()
    else:
        acquisition_timestamp = ''
    return session_timestamp, acquisition_timestamp


def get_sex_string(sex_str):
    """
    Return male or female string.
    """
    if sex_str == 'M':
        sex = 'male'
    elif sex_str == 'F':
        sex = 'female'
    else:
        sex = ''
    return sex


def assign_type(s):
    """
    Sets the type of a given input.
    """
    if type(s) == pydicom.valuerep.PersonName or type(s) == pydicom.valuerep.PersonName3 or type(s) == pydicom.valuerep.PersonNameBase:
        return format_string(s)
    if type(s) == list or type(s) == pydicom.multival.MultiValue:
        try:
            return [ int(x) for x in s ]
        except ValueError:
            try:
                return [ float(x) for x in s ]
            except ValueError:
                return [ format_string(x) for x in s if len(x) > 0 ]
    else:
        s = str(s)
        try:
            return int(s)
        except ValueError:
            try:
                return float(s)
            except ValueError:
                return format_string(s)


def format_string(in_string):
    formatted = re.sub(r'[^\x00-\x7f]',r'', str(in_string)) # Remove non-ascii characters
    formatted = ''.join(filter(lambda x: x in string.printable, formatted))
    if len(formatted) == 1 and formatted == '?':
        formatted = None
    return formatted#.encode('utf-8').strip()


def get_seq_data(sequence, ignore_keys):
    seq_dict = {}
    for seq in sequence:
        for s_key in seq.dir():
            s_val = getattr(seq, s_key, '')
            if type(s_val) is pydicom.UID.UID or s_key in ignore_keys:
                continue

            if type(s_val) == pydicom.sequence.Sequence:
                _seq = get_seq_data(s_val, ignore_keys)
                seq_dict[s_key] = _seq
                continue

            if type(s_val) == str:
                s_val = format_string(s_val)
            else:
                s_val = assign_type(s_val)

            if s_val:
                seq_dict[s_key] = s_val

    return seq_dict


def get_pydicom_header(dcm):
    # Extract the header values
    header = {}
    exclude_tags = ['[Unknown]', 
                    'PixelData', 
                    'Pixel Data',  
                    '[User defined data]', 
                    '[Protocol Data Block (compressed)]', 
                    '[Histogram tables]', 
                    '[Unique image iden]']
    tags = dcm.dir()
    for tag in tags:
        try:
            if (tag not in exclude_tags) and ( type(dcm.get(tag)) != pydicom.sequence.Sequence ):
                value = dcm.get(tag)
                if value or value == 0: # Some values are zero
                    # Put the value in the header
                    if type(value) == str and len(value) < 10240: # Max pydicom field length
                        header[tag] = format_string(value)
                    else:
                        header[tag] = assign_type(value)
                else:
                    log.debug('No value found for tag: ' + tag)

            if type(dcm.get(tag)) == pydicom.sequence.Sequence:
                seq_data = get_seq_data(dcm.get(tag), exclude_tags)
                # Check that the sequence is not empty
                if seq_data:
                    header[tag] = seq_data
        except:
            log.debug('Failed to get ' + tag)
            pass
    return header


def get_csa_header(dcm):
    exclude_tags = ['PhoenixZIP', 'SrMsgBuffer']
    header = {}
    try:
        raw_csa_header = nibabel.nicom.dicomwrappers.SiemensWrapper(dcm).csa_header
        tags = raw_csa_header['tags']
    except:
        log.warning('Failed to parse csa header!')
        return header

    for tag in tags:
        if not raw_csa_header['tags'][tag]['items'] or tag in exclude_tags:
            log.debug('Skipping : %s' % tag)
            pass
        else:
            value = raw_csa_header['tags'][tag]['items']
            if len(value) == 1:
                value = value[0]
                if type(value) == str and ( len(value) > 0 and len(value) < 1024 ):
                    header[format_string(tag)] = format_string(value)
                else:
                    header[format_string(tag)] = assign_type(value)
            else:
                header[format_string(tag)] = assign_type(value)

    return header


def get_classification_from_string(value):
    result = {}

    parts = re.split(r'\s*,\s*', value)
    last_key = None
    for part in parts:
        key_value = re.split(r'\s*:\s*', part)

        if len(key_value) == 2:
            last_key = key = key_value[0]
            value = key_value[1]
        else:
            if last_key:
                key = last_key
            else:
                log.warning('Unknown classification format: {0}'.format(part))
                key = 'Custom'
            value = part

        if key not in result:
            result[key] = []

        result[key].append(value)

    return result


def validate_against_template(input_dict, template, error_log_path):
    """
    This is a function for validating a dictionary against a template. Given
    an input_dict and a template object, it will create a JSON schema validator
    and construct an object that is a list of error dictionaries. It will write a
    JSON file to the specified error_log_path and return the validation_errors object as
    well as log each error.message to log.errors

    :param input_dict: a dictionary of DICOM header data to be validated
    :param template: a template dictionary to validate against
    :param error_log_path: the path to which to write error log JSON
    :return: validation_errors, an object containing information on validation errors
    """
    # Initialize json schema validator
    validator = jsonschema.Draft7Validator(template)
    # Initialize list object for storing validation errors
    validation_errors = []
    for error in sorted(validator.iter_errors(input_dict), key=str):
        # Create a temporary dictionary for the individual error
        tmp_dict = {}
        # Get error type
        tmp_dict['error_type'] = error.validator
        # Get error message and log it
        tmp_dict['error_message'] = error.message
        log.error(error.message)
        # Required field errors are a little special and need to be handled
        # separately to get the field. We don't get the schema because it
        # will print the entire template schema
        if error.validator == "required":
            # Get the item failing validation from the error message
            tmp_dict['item'] = 'info.' + error.message.split("'")[1]
        # Get additional information for pattern and type errors
        elif error.validator in ("pattern", "type"):
            # Get the value of the field that failed validation
            tmp_dict['error_value'] = error.instance
            # Get the field that failed validation
            tmp_dict['item'] = 'info.' + str(error.path.pop())
            # Get the schema object used to validate in failed validation
            tmp_dict['schema'] = error.schema
        elif error.validator == "anyOf":
            tmp_dict['schema'] = {"anyOf": error.schema['anyOf']}
        else:
            pass
        # Append individual error object to the return validation_errors object
        validation_errors.append(tmp_dict)

    with open(error_log_path, 'w') as outfile:
        json.dump(validation_errors, outfile, separators=(', ', ': '), sort_keys=True, indent=4)
    return validation_errors


def dicom_to_json(zip_file_path, outbase, timezone):
    # Check for input file path
    if not os.path.exists(zip_file_path):
        log.debug('could not find %s' % zip_file_path)
        log.debug('checking input directory ...')
        if os.path.exists(os.path.join('/input', zip_file_path)):
            zip_file_path = os.path.join('/input', zip_file_path)
            log.debug('found %s' % zip_file_path)

    if not outbase:
        outbase = '/flywheel/v0/output'
        log.info('setting outbase to %s' % outbase)

    # Extract the last file in the zip to /tmp/ and read it
    dcm = []
    if zipfile.is_zipfile(zip_file_path):
        zip = zipfile.ZipFile(zip_file_path)
        num_files = len(zip.namelist())
        for n in range((num_files - 1), -1, -1):
            dcm_path = zip.extract(zip.namelist()[n], '/tmp')
            if os.path.isfile(dcm_path):
                try:
                    log.info('reading %s' % dcm_path)
                    dcm = pydicom.read_file(dcm_path)
                    # Here we check for the Raw Data Storage SOP Class, if there
                    # are other pydicom files in the zip then we read the next one,
                    # if this is the only class of pydicom in the file, we accept
                    # our fate and move on.
                    if dcm.get('SOPClassUID') == 'Raw Data Storage' and n != range((num_files - 1), -1, -1)[-1]:
                        continue
                    else:
                        break
                except:
                    pass
            else:
                log.warning('%s does not exist!' % dcm_path)
    else:
        log.info('Not a zip. Attempting to read %s directly' % os.path.basename(zip_file_path))
        dcm = pydicom.read_file(zip_file_path)

    if not dcm:
        log.warning('dcm is empty!!!')
        os.sys.exit(1)

    # Build metadata
    metadata = {}

    # Session metadata
    metadata['session'] = {}
    session_timestamp, acquisition_timestamp = get_timestamp(dcm, timezone);
    if session_timestamp:
        metadata['session']['timestamp'] = session_timestamp
    if hasattr(dcm, 'OperatorsName') and dcm.get('OperatorsName'):
        metadata['session']['operator'] = format_string(dcm.get('OperatorsName'))
    session_label = get_session_label(dcm)
    if session_label:
        metadata['session']['label'] = session_label

    # Subject Metadata
    metadata['session']['subject'] = {}
    if hasattr(dcm, 'PatientSex') and get_sex_string(dcm.get('PatientSex')):
        metadata['session']['subject']['sex'] = get_sex_string(dcm.get('PatientSex'))
    if hasattr(dcm, 'PatientAge') and dcm.get('PatientAge'):
        try:
            age = parse_patient_age(dcm.get('PatientAge'))
            if age:
                metadata['session']['subject']['age'] = int(age)
        except:
            pass
    if hasattr(dcm, 'PatientName') and dcm.get('PatientName').given_name:
        # If the first name or last name field has a space-separated string, and one or the other field is not
        # present, then we assume that the operator put both first and last names in that one field. We then
        # parse that field to populate first and last name.
        metadata['session']['subject']['firstname'] = str(format_string(dcm.get('PatientName').given_name))
        if not dcm.get('PatientName').family_name:
            name = format_string(dcm.get('PatientName').given_name.split(' '))
            if len(name) == 2:
                first = name[0]
                last = name[1]
                metadata['session']['subject']['lastname'] = str(last)
                metadata['session']['subject']['firstname'] = str(first)
    if hasattr(dcm, 'PatientName') and dcm.get('PatientName').family_name:
        metadata['session']['subject']['lastname'] = str(format_string(dcm.get('PatientName').family_name))
        if not dcm.get('PatientName').given_name:
            name = format_string(dcm.get('PatientName').family_name.split(' '))
            if len(name) == 2:
                first = name[0]
                last = name[1]
                metadata['session']['subject']['lastname'] = str(last)
                metadata['session']['subject']['firstname'] = str(first)

    # File classification
    pydicom_file = {}
    pydicom_file['name'] = os.path.basename(zip_file_path)
    pydicom_file['modality'] = format_string(dcm.get('Modality', 'MR'))

    # Acquisition metadata
    metadata['acquisition'] = {}
    if hasattr(dcm, 'Modality') and dcm.get('Modality'):
        metadata['acquisition']['instrument'] = format_string(dcm.get('Modality'))

    series_desc = format_string(dcm.get('SeriesDescription', ''))
    if series_desc:
        metadata['acquisition']['label'] = series_desc

    if acquisition_timestamp:
        metadata['acquisition']['timestamp'] = acquisition_timestamp

    # Acquisition metadata from pydicom header
    pydicom_file['info'] = get_pydicom_header(dcm)

    # Validate header data
    error_filepath = os.path.join(output_folder, 'error.log.json')
    validation_errors = validate_against_template(pydicom_file['info'], json_template, error_filepath)
    if validation_errors:
        metadata['acquisition']['tags'] = ['error']

    # Append the pydicom_file to the files array
    metadata['acquisition']['files'] = [pydicom_file]

    # Acquisition metadata from pydicom header
    metadata['acquisition']['metadata'] = get_pydicom_header(dcm)
    if dcm.get('Manufacturer') == 'SIEMENS':
        csa_header = get_csa_header(dcm)
        if csa_header:
            metadata['acquisition']['metadata']['CSAHeader'] = csa_header

    # Write out the metadata to file (.metadata.json)
    metafile_outname = os.path.join(os.path.dirname(outbase), '.metadata.json')
    with open(metafile_outname, 'w') as metafile:
        json.dump(metadata, metafile, separators=(', ', ': '), sort_keys=True, indent=4)

    # Show the metadata
    pprint(metadata)

    return metafile_outname

## Load in JSON Schema template

In [75]:
template_filepath = "U2787g_template"

with open(template_filepath) as template_data:
    template_json = json.load(template_data)

In [76]:
test_dictionary = {
    "Modality": "CT",
    "ImageType": ["SCREEN SAVE","OTHER"],
    "PatientID": "55555",
}

In [77]:
validate_against_template(test_dictionary, template_json, "error.log.json")

ERROR:U2787g:'CT' does not match 'MR'
ERROR:U2787g:{'Modality': 'CT', 'ImageType': ['SCREEN SAVE', 'OTHER'], 'PatientID': '55555'} is not valid under any of the given schemas
ERROR:U2787g:{'enum': ['SCREEN SAVE']} is not allowed for 'SCREEN SAVE'


[{'error_type': 'pattern',
  'error_message': "'CT' does not match 'MR'",
  'error_value': 'CT',
  'item': 'info.Modality',
  'schema': {'description': "Modality must match 'MR'",
   'pattern': 'MR',
   'type': 'string'}},
 {'error_type': 'anyOf',
  'error_message': "{'Modality': 'CT', 'ImageType': ['SCREEN SAVE', 'OTHER'], 'PatientID': '55555'} is not valid under any of the given schemas",
  'schema': {'anyOf': [{'required': ['AcquisitionDate']},
    {'required': ['SeriesDate']},
    {'required': ['StudyDate']}]}},
 {'error_type': 'not',
  'error_message': "{'enum': ['SCREEN SAVE']} is not allowed for 'SCREEN SAVE'"}]

In [116]:
template_filepath = "U2787g_template"
# Configure timezone and dicom filepath
timezone = validate_timezone(tzlocal.get_localzone())
zip_file_path = "A.zip"
outbase = os.getcwd()
output_folder = os.getcwd()
# Import JSON template
with open(template_filepath) as template_data:
    json_template = json.load(template_data)


In [117]:
# Check for input file path
if not os.path.exists(zip_file_path):
    log.debug('could not find %s' % zip_file_path)
    log.debug('checking input directory ...')
    if os.path.exists(os.path.join('/input', zip_file_path)):
        zip_file_path = os.path.join('/input', zip_file_path)
        log.debug('found %s' % zip_file_path)

if not outbase:
    outbase = '/flywheel/v0/output'
    log.info('setting outbase to %s' % outbase)

# Extract the last file in the zip to /tmp/ and read it
dcm = []
if zipfile.is_zipfile(zip_file_path):
    zip = zipfile.ZipFile(zip_file_path)
    num_files = len(zip.namelist())
    for n in range((num_files - 1), -1, -1):
        dcm_path = zip.extract(zip.namelist()[n], '/tmp')
        if os.path.isfile(dcm_path):
            try:
                log.info('reading %s' % dcm_path)
                dcm = pydicom.read_file(dcm_path)
                # Here we check for the Raw Data Storage SOP Class, if there
                # are other pydicom files in the zip then we read the next one,
                # if this is the only class of pydicom in the file, we accept
                # our fate and move on.
                if dcm.get('SOPClassUID') == 'Raw Data Storage' and n != range((num_files - 1), -1, -1)[-1]:
                    continue
                else:
                    break
            except:
                pass
        else:
            log.warning('%s does not exist!' % dcm_path)
else:
    log.info('Not a zip. Attempting to read %s directly' % os.path.basename(zip_file_path))
    dcm = pydicom.read_file(zip_file_path)

if not dcm:
    log.warning('dcm is empty!!!')
    os.sys.exit(1)

# Build metadata
metadata = {}

# Session metadata
metadata['session'] = {}
session_timestamp, acquisition_timestamp = get_timestamp(dcm, timezone);
if session_timestamp:
    metadata['session']['timestamp'] = session_timestamp
if hasattr(dcm, 'OperatorsName') and dcm.get('OperatorsName'):
    metadata['session']['operator'] = format_string(dcm.get('OperatorsName'))
session_label = get_session_label(dcm)
if session_label:
    metadata['session']['label'] = session_label

# Subject Metadata
metadata['session']['subject'] = {}
if hasattr(dcm, 'PatientSex') and get_sex_string(dcm.get('PatientSex')):
    metadata['session']['subject']['sex'] = get_sex_string(dcm.get('PatientSex'))
if hasattr(dcm, 'PatientAge') and dcm.get('PatientAge'):
    try:
        age = parse_patient_age(dcm.get('PatientAge'))
        if age:
            metadata['session']['subject']['age'] = int(age)
    except:
        pass
if hasattr(dcm, 'PatientName') and dcm.get('PatientName').given_name:
    # If the first name or last name field has a space-separated string, and one or the other field is not
    # present, then we assume that the operator put both first and last names in that one field. We then
    # parse that field to populate first and last name.
    metadata['session']['subject']['firstname'] = str(format_string(dcm.get('PatientName').given_name))
    if not dcm.get('PatientName').family_name:
        name = format_string(dcm.get('PatientName').given_name.split(' '))
        if len(name) == 2:
            first = name[0]
            last = name[1]
            metadata['session']['subject']['lastname'] = str(last)
            metadata['session']['subject']['firstname'] = str(first)
if hasattr(dcm, 'PatientName') and dcm.get('PatientName').family_name:
    metadata['session']['subject']['lastname'] = str(format_string(dcm.get('PatientName').family_name))
    if not dcm.get('PatientName').given_name:
        name = format_string(dcm.get('PatientName').family_name.split(' '))
        if len(name) == 2:
            first = name[0]
            last = name[1]
            metadata['session']['subject']['lastname'] = str(last)
            metadata['session']['subject']['firstname'] = str(first)

# File classification
pydicom_file = {}
pydicom_file['name'] = os.path.basename(zip_file_path)
pydicom_file['modality'] = format_string(dcm.get('Modality', 'MR'))

# Acquisition metadata
metadata['acquisition'] = {}
if hasattr(dcm, 'Modality') and dcm.get('Modality'):
    metadata['acquisition']['instrument'] = format_string(dcm.get('Modality'))

series_desc = format_string(dcm.get('SeriesDescription', ''))
if series_desc:
    metadata['acquisition']['label'] = series_desc

if acquisition_timestamp:
    metadata['acquisition']['timestamp'] = acquisition_timestamp

# Acquisition metadata from pydicom header
pydicom_file['info'] = get_pydicom_header(dcm)

# Validate header data
error_filepath = os.path.join(output_folder, 'error.log.json')
validation_errors = validate_against_template(pydicom_file['info'], json_template, error_filepath)
if validation_errors:
    metadata['acquisition']['tags'] = ['error']

# Append the pydicom_file to the files array
metadata['acquisition']['files'] = [pydicom_file]

# Acquisition metadata from pydicom header
metadata['acquisition']['metadata'] = get_pydicom_header(dcm)
if dcm.get('Manufacturer') == 'SIEMENS':
    csa_header = get_csa_header(dcm)
    if csa_header:
        metadata['acquisition']['metadata']['CSAHeader'] = csa_header

# Write out the metadata to file (.metadata.json)
metafile_outname = os.path.join(os.path.dirname(outbase), '.metadata.json')
with open(metafile_outname, 'w') as metafile:
    json.dump(metadata, metafile, separators=(', ', ': '), sort_keys=True, indent=4)

# Show the metadata
pprint(metadata)

ERROR:U2787g:'737178' does not match '^[0-9]{5}$'


{'acquisition': {'files': [{'info': {'AccessionNumber': '039208195',
                                     'AcquisitionDate': '20050623',
                                     'AcquisitionMatrix': [0, 256, 256, 0],
                                     'AcquisitionNumber': 1,
                                     'AcquisitionTime': '083051',
                                     'AdditionalPatientHistory': 'RESEARCH PT',
                                     'AngioFlag': 'N',
                                     'BitsAllocated': 16,
                                     'BitsStored': 12,
                                     'CardiacNumberOfImages': 0,
                                     'Columns': 256,
                                     'ContentDate': '20050623',
                                     'ContentTime': '083051',
                                     'DeviceSerialNumber': '000000209526SMMR',
                                     'EchoNumbers': 1,
                                  

## Import all dcm file headers

I'd like a way to import all headers as a dataframe in order to do validation across files. Pandas does not accept lists as dictionary values, so just non-sequence for now.


In [118]:
def assign_type_pd(s):
    """
    Sets the type of a given input.
    """
    if type(s) == pydicom.valuerep.PersonName or type(s) == pydicom.valuerep.PersonName3 or type(s) == pydicom.valuerep.PersonNameBase:
        return format_string(s)
    else:
        s = str(s)
        try:
            return int(s)
        except ValueError:
            try:
                return float(s)
            except ValueError:
                return format_string(s)
            
            
def dicom_header_to_pd(dcm):
    # Extract the header values
    header = {}
    exclude_tags = ['[Unknown]', 
                    'PixelData', 
                    'Pixel Data',  
                    '[User defined data]', 
                    '[Protocol Data Block (compressed)]', 
                    '[Histogram tables]', 
                    '[Unique image iden]']
    tags = dcm.dir()
    for tag in tags:
        try:
            if (tag not in exclude_tags) and ( type(dcm.get(tag)) != pydicom.sequence.Sequence ):
                value = dcm.get(tag)
                if value or value == 0: # Some values are zero
                    # Put the value in the header
                    if type(value) == str and len(value) < 10240: # Max pydicom field length
                        header[tag] = [format_string(value)]
                    else:
                        header[tag] = [assign_type_pd(value)]
                        
                else:
                    log.debug('No value found for tag: ' + tag)
        except:
            log.debug('Failed to get ' + tag)
            pass
    dataframe = pd.DataFrame.from_dict(header)
    return dataframe
    

,AccessionNumber,AcquisitionDate,AcquisitionMatrix,AcquisitionNumber,AcquisitionTime,AdditionalPatientHistory,AngioFlag,BitsAllocated,BitsStored,CardiacNumberOfImages,...,StationName,StudyDate,StudyDescription,StudyID,StudyInstanceUID,StudyTime,TriggerWindow,VariableFlipAngleFlag,WindowCenter,WindowWidth
0,039208195,20050623,"[0, 256, 256, 0]",1,083051,RESEARCH PT,N,16,12,0,...,GEMSOW,20050623,BRAIN,21671,1.2.840.113619.2.176.3596.6688930.8424.1119539...,082053,0,N,546,1093


In [364]:
zip_file_path = 'A.zip'
dcm_list = []
if zipfile.is_zipfile(zip_file_path):
    zip = zipfile.ZipFile(zip_file_path)
    num_files = len(zip.namelist())
    for n in range((num_files - 1), -1, -1):
        dcm_path = zip.extract(zip.namelist()[n], '/tmp')
        dcm_tmp = None
        if os.path.isfile(dcm_path):
            try:
                log.info('reading %s' % dcm_path)
                dcm_tmp = pydicom.read_file(dcm_path)
                # Here we check for the Raw Data Storage SOP Class, if there
                # are other pydicom files in the zip then we read the next one,
                # if this is the only class of pydicom in the file, we accept
                # our fate and move on.
                if dcm_tmp.get('SOPClassUID') == 'Raw Data Storage' and n != range((num_files - 1), -1, -1)[-1]:
                    continue
                else:
                    dcm_list.append(dcm_tmp)
            except:
                pass
        else:
            log.warning('%s does not exist!' % dcm_path)
else:
    log.info('Not a zip. Attempting to read %s directly' % os.path.basename(zip_file_path))
    dcm = pydicom.read_file(zip_file_path)
dcm = dcm_list[-1]
if not dcm:
    log.warning('dcm is empty!!!')
    os.sys.exit(1)

In [ ]:
for dcm in dcm_list

In [162]:
df_loc = dicomzip_to_dataframe("4784_1_1_loc.dcm.zip")
df_loc.to_csv('4784_1_1_loc.csv')

In [144]:
df_A = dicomzip_to_dataframe("A.zip")
df_A['ImagePositionPatient']
df_t1 = dicomzip_to_dataframe("/Users/kalebfischer/Downloads/4784_3_1_t1.dcm.zip")
df_t2 = dicomzip_to_dataframe("/Users/kalebfischer/Downloads/4784_5_1_fmri.dcm.zip")

In [173]:
for df in [df_A,df_loc,df_t1,df_t2]:
    print("\n")
    print(df['ImagesInAcquisition'].unique())
    print(df.shape)



[53]
(25, 88)


[14]
(14, 88)


[36]
(36, 90)


[5880]
(350, 93)


In [179]:
df['SeriesInstanceUID'].unique()

array(['1.2.840.113619.2.283.4120.7575399.23776.1370277200.2'],
      dtype=object)

In [370]:
#print(df.nunique())
for col in df.columns:
    if len(df[col].unique()) > 1:
        print(df[col])

0     [-118, -130, -52]
1     [-118, -130, -40]
2     [-118, -130, -46]
3     [-118, -130, -58]
4     [-118, -130, -88]
5      [-118, -130, 43]
6      [-118, -130, 49]
7     [-118, -130, -22]
8      [-118, -130, -4]
9     [-118, -130, -10]
10    [-118, -130, -16]
11     [-118, -130, 55]
12     [-118, -130, 37]
13    [-118, -130, -70]
14    [-118, -130, -76]
15    [-118, -130, -82]
16    [-118, -130, -64]
17     [-118, -130, 13]
18     [-118, -130, 25]
19      [-118, -130, 1]
20    [-118, -130, -28]
21    [-118, -130, -34]
22      [-118, -130, 7]
23     [-118, -130, 31]
24     [-118, -130, 19]
Name: ImagePositionPatient, dtype: object
0     13
1     17
2     15
3     11
4      1
5     45
6     47
7     23
8     29
9     27
10    25
11    49
12    43
13     7
14     5
15     3
16     9
17    35
18    39
19    31
20    21
21    19
22    33
23    41
24    37
Name: InstanceNumber, dtype: int64
0     1093
1      902
2     1005
3     1001
4      763
5      868
6      836
7      923
8      925

In [378]:
def classify_dicom(dcm,slice_number,uniqueIOP):
    TR = dcm.get('RepetitionTime')
    TE = dcm.get('EchoTime')
    TI = dcm.get('InversionTime')
    SD = dcm.get('SeriesDescription')
    classification_dict = {}
    if TE < 30 and TR < 800:
        classification_dict['Measurement'] = ["T1"]
    elif TE > 50 and TR > 2000 and TI == 0:
        classification_dict['Measurement'] = ["T2"]
    elif TE > 50 and TR > 8000 and (3000 > TI > 1500):
        classification_dict['Measurement'] = ["FLAIR"]
    elif TE < 50 and TR > 1000:
        classification_dict['Measurement'] = ["PD"]
    if re.search('POST', SD, flags=re.IGNORECASE):
        classification_dict['Custom'] = ['Contrast']
    if slice_number < 10:
        classification_dict['Intent'] = ['Localizer']
    if uniqueIOP:
        classification_dict['Intent'] = ['3-Plane Localizer']
    return classification_dict


In [379]:
slice_number = len(df) 
uniqueIOP = df.ImageOrientationPatient.is_unique
classify_dicom(dcm,slice_number,uniqueIOP)

{'Measurement': ['T2']}

In [352]:
dcm.get('SeriesDescription')

'3Plane Loc SSFSE'

In [361]:
df.ImageOrientationPatient.is_unique

False

In [380]:
default_template = {
  "properties": {
    "ImageType": {
            "description": "ImageType cannot be 'SCREEN SAVE'",
            "type": "array",
            "items": {
              "not": {"enum":["SCREEN SAVE"]}
            }
        },
    "Modality": {
            "description": "Modality must match 'MR'",
            "pattern": "MR",
            "type": "string"
        },
    "PatientID": {
            "description": "PatientID must be 5 numeric characters",
            "pattern": "^[0-9]{6}$",
            "type": "string"
        }
  },
  "type": "object",
  "anyOf": [
    {"required": ["AcquisitionDate"]},
    {"required": ["SeriesDate"]},
    {"required": ["StudyDate"]}
  ]
}

In [400]:
df['InstanceNumber'].duplicated()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
Name: InstanceNumber, dtype: bool

yes


In [405]:

duplicates = df.loc[df['ImageOrientationPatient'].duplicated(),'InstanceNumber'].values
test_dict = {
    "error_message": "InstanceNumber is duplicated for values:{}".format(duplicates),
    "revalidate": False
}
test_dict

{'error_message': 'InstanceNumber is duplicated for values:[17 15 11  1 45 47 23 29 27 25 49 43  7  5  3  9 35 39 31 21 19 33 41 37]',
 'revalidate': False}

In [415]:
def validate_against_rules(df):
    error_list = []
    # Determine if InstanceNumber is unique (not duplicated)
    if 'InstanceNumber' in df:
        if df['InstanceNumber'].is_unique:
            pass
        else:
            duplicated_values = df.loc[df['InstanceNumber'].duplicated(),'InstanceNumber'].values
            error_dict = {
                "error_message": "InstanceNumber is duplicated for values:{}".format(duplicated_values),
                "revalidate": False
            }
            error_list.append(error_dict)

    if 'ImagesInAcquisition' in df:
        expected_images = df['ImagesInAcquisition'][0]
    elif 'NumberOfSeriesRelatedInstances' in df:
        expected_images = df['NumberOfSeriesRelatedInstances'][0]
    elif 'NumberOfSlices' in df:
        expected_images = df['NumberOfSlices']
    else:
        error_dict = {
                "error_message": "Could not locate a DICOM header field for expected number of images.",
                "revalidate": False
        }
        error_list.append(error_dict)
    found_images = len(df)
    if expected_images != found_images:
        error_dict = {
                "error_message": "Expected {} DICOM files, found {}".format(expected_images, found_images),
                "revalidate": False
        }
    return error_list

In [429]:
def dicom_date_handler(dcm):
    if dcm.get('AcquisitionDate'):
        pass
    elif dcm.get('SeriesDate'):
        dcm.AcquisitionDate = dcm.get('SeriesDate')
    elif dcm.get('StudyDate'):
        dcm.AcquisitionDate = dcm.get('StudyDate')
    else: 
        log.warning('No date found for DICOM file')
    return dcm

In [423]:
date = dcm.AcquisitionDate


In [426]:
dcm.AcquisitionDate = date

In [87]:
TE = 52
TI = 3800
TR = 8100

In [94]:

if TE < 30 and TR < 800:
    measurement = "T1"
elif TE > 50 and TR > 2000 and TI == 0:
    measurement = "T2"
elif TE > 50 and TR > 8000 and (3000 > TI > 1500):
    measurement = "FLAIR"
elif TE < 50 and TR > 1000:
    measurement = "PD"
else:
    measurement = None
print(measurement)

T2


In [113]:
df.ImageOrientationPatient.is_unique


False

## Refactor Classifier
I'd like to refactor the classifier code for generating metadata to be more modular and extensible


In [306]:
# define a blacklist for DICOM header tags
DICOM_TAG_BLACKLIST = ['[Unknown]',
                       'PixelData',
                       'Pixel Data',
                       '[User defined data]',
                       '[Protocol Data Block (compressed)]',
                       '[Histogram tables]',
                       '[Unique image iden]']


def read_dicom_header(dicom_filepath):
    dcm = pydicom.read_file(dicom_filepath)
    
df_list = []
dcm_list = []
if zipfile.is_zipfile(zip_file_path):
    zip = zipfile.ZipFile(zip_file_path)
    for zip_file in zip.namelist():
        dcm_path = zip.extract(zip_file, '/tmp')
        if os.path.isfile(dcm_path):
            log.info('reading %s' % dcm_path)
            dcm_list.append(pydicom.read_file(dcm_path))
            # Here we check for the Raw Data Storage SOP Class, if there
            # are other pydicom files in the zip then we read the next one,
            # if this is the only class of pydicom in the file, we accept
            # our fate and move on.
            if dcm.get('SOPClassUID') == 'Raw Data Storage' and n != range((num_files - 1), -1, -1)[-1]:
                continue
dcm = dcm

In [366]:
df_list = []
for header in dcm_list:
    tmp_dict = get_pydicom_header(header)
    for key in tmp_dict:
        if type(tmp_dict[key]) == list:
            tmp_dict[key] = str(tmp_dict[key])
        else:
            tmp_dict[key] = [tmp_dict[key]] 
    df_tmp = pd.DataFrame.from_dict(tmp_dict)
    df_list.append(df_tmp)
df = pd.concat(df_list,ignore_index=True)

In [338]:
print (df.loc[:, df.apply(lambda x: x.nunique()) > 1].columns)

Index(['ImagePositionPatient', 'InstanceNumber', 'LargestImagePixelValue',
       'SOPInstanceUID', 'SliceLocation', 'WindowCenter', 'WindowWidth'],
      dtype='object')


In [343]:
if not df.SliceLocation.is_unique:
    print("unique")

In [187]:
header = {}
dcm = pydicom.read_file(dcm_path) 
for element in dcm.dir():
    if (element not in DICOM_TAG_BLACKLIST) and (type(dcm.get(tag)) != pydicom.sequence.Sequence):
        header[element] = dcm.get(element)

In [188]:
header

{'AccessionNumber': '',
 'AcquisitionDate': '20130604',
 'AcquisitionMatrix': [0, 256, 128, 0],
 'AcquisitionNumber': "1",
 'AcquisitionTime': '223626',
 'AdditionalPatientHistory': '',
 'AngioFlag': 'N',
 'BitsAllocated': 16,
 'BitsStored': 16,
 'CardiacNumberOfImages': "0",
 'Columns': 256,
 'ContentDate': '20130604',
 'ContentTime': '223626',
 'DeviceSerialNumber': '00000650723PSYMR',
 'EchoNumbers': "1",
 'EchoTime': "1.728",
 'EchoTrainLength': "1",
 'FlipAngle': "30",
 'FrameOfReferenceUID': '1.2.840.113619.2.283.4120.7575399.23776.1370277199.998',
 'HeartRate': "0",
 'HighBit': 15,
 'ImageOrientationPatient': ['-0', '1', '0', '-0', '-0', '-1'],
 'ImagePositionPatient': ['-32.5', '-139.531', '119.531'],
 'ImageType': ['ORIGINAL', 'PRIMARY', 'OTHER'],
 'ImagedNucleus': '1H',
 'ImagesInAcquisition': "14",
 'ImagingFrequency': "127.679822",
 'InPlanePhaseEncodingDirection': 'ROW',
 'InstanceNumber': "14",
 'InstitutionName': 'Institution Name',
 'InversionTime': "0",
 'LargestImageP

In [190]:
modified_header = get_pydicom_header(dcm)

In [204]:
for key in modified_header:
    print(key)
    print(type(dcm.get(key)))
    print(modified_header[key])
    print(header[key])
    print("\n")

AcquisitionDate
<class 'str'>
20130604
20130604


AcquisitionMatrix
<class 'list'>
[0, 256, 128, 0]
[0, 256, 128, 0]


AcquisitionNumber
<class 'pydicom.valuerep.IS'>
1
1


AcquisitionTime
<class 'str'>
223626
223626


AngioFlag
<class 'str'>
N
N


BitsAllocated
<class 'int'>
16
16


BitsStored
<class 'int'>
16
16


CardiacNumberOfImages
<class 'pydicom.valuerep.IS'>
0
0


Columns
<class 'int'>
256
256


ContentDate
<class 'str'>
20130604
20130604


ContentTime
<class 'str'>
223626
223626


DeviceSerialNumber
<class 'str'>
00000650723PSYMR
00000650723PSYMR


EchoNumbers
<class 'pydicom.valuerep.IS'>
1
1


EchoTime
<class 'pydicom.valuerep.DSfloat'>
1.728
1.728


EchoTrainLength
<class 'pydicom.valuerep.IS'>
1
1


FlipAngle
<class 'pydicom.valuerep.DSfloat'>
30
30


FrameOfReferenceUID
<class 'pydicom.uid.UID'>
1.2.840.113619.2.283.4120.7575399.23776.1370277199.998
1.2.840.113619.2.283.4120.7575399.23776.1370277199.998


HeartRate
<class 'pydicom.valuerep.IS'>
0
0


HighBit
<class 'int'

In [197]:
type(dcm.get('RepetitionTime'))

pydicom.valuerep.DSfloat

In [203]:
s = dcm.get('PixelSpacing')
[ float(x) for x in s ]

[0.9375, 0.9375]

In [301]:
FLOAT_VR = [
    "FL",
    "FD",
    "DS",
    "OF"
]

INT_VR = [
    "IS",
    "SL",
    "SS",
    "UL",
    "US"
]

In [305]:
for element in dcm:
    if element.keyword:
        key = element.keyword
        value = element.value
        value_type = type(element.value)
        vr = element.VR
        if vr in (FLOAT_VR or INT_VR):
            if value_type not in [list, pydicom.multival.MultiValue]:
                value = number_parser(str(value))
            else:
                value = [number_parser(str(x)) for x in value]
        
        print(element.keyword)
        print(element.VR)
        print(value)
        print("__________________________")

SpecificCharacterSet
CS
ISO_IR 100
__________________________
ImageType
CS
['ORIGINAL', 'PRIMARY', 'OTHER']
__________________________
SOPClassUID
UI
1.2.840.10008.5.1.4.1.1.4
__________________________
SOPInstanceUID
UI
1.2.840.113619.2.283.4120.7575399.13868.1370277913.340
__________________________
StudyDate
DA
20130604
__________________________
SeriesDate
DA
20130604
__________________________
AcquisitionDate
DA
20130604
__________________________
ContentDate
DA
20130604
__________________________
StudyTime
TM
223620
__________________________
SeriesTime
TM
223626
__________________________
AcquisitionTime
TM
223626
__________________________
ContentTime
TM
223626
__________________________
AccessionNumber
SH

__________________________
Modality
CS
MR
__________________________
Manufacturer
LO
GE MEDICAL SYSTEMS
__________________________
InstitutionName
LO
Institution Name
__________________________
ReferringPhysicianName
PN

__________________________
StationName
SH
MRIPSMR
____

In [246]:
def assign_type(s):
    """
    Sets the type of a given input.
    """
    if type(s) == dicom.valuerep.PersonName or type(s) == dicom.valuerep.PersonName3 or type(s) == dicom.valuerep.PersonNameBase:
        return format_string(s)
    if type(s) == list or type(s) == dicom.multival.MultiValue:
        try:
            return [ int(x) for x in s ]
        except ValueError:
            try:
                return [ float(x) for x in s ]
            except ValueError:
                return [ format_string(x) for x in s if len(x) > 0 ]
    else:
        s = str(s)
        try:
            return int(s)
        except ValueError:
            try:
                return float(s)
            except ValueError:
                return format_string(s)

str

In [292]:

def number_parser(input_float):
    '''
    Takes a string representation of a number and returns 
    '''
    from ast import literal_eval
    try:
        val = literal_eval(input_float)
    except ValueError:
        log.error(' \'{}\' is neither integer nor float. Returning as is.'.format(input_float))
        return(input_float)
    if isinstance(val, int) or (isinstance(val, float) and val.is_integer()):
        return int(val)
    elif isinstance(val, float):
        return float(val)
    

In [225]:
def format_string(in_string):
    formatted = re.sub(r'[^\x00-\x7f]',r'', str(in_string)) # Remove non-ascii characters
    formatted = ''.join(filter(lambda x: x in string.printable, formatted))
    if len(formatted) == 1 and formatted == '?':
        formatted = None
    return formatted

'M'

In [272]:
stringthing = ''
''.join(filter(lambda x: x in string.printable, stringthing))

''

In [281]:
#def pydicom_to_dict(dcm):
header_dict = {}
for element in dcm:
    if element.keyword and (element.keyword not in DICOM_TAG_BLACKLIST):
        key = element.keyword
        value = element.value
        value_type = type(element.value)
        vr = element.VR
       
        if value_type == str:
            value = format_string(value)
        elif type(element.value) == pydicom.sequence.Sequence: 
            print('Sequence')

        elif type(element.value) == pydicom.multival.MultiValue:
            # Get the type for the elements of the MultiValue object
            multival_type = dcm.get(element.keyword).type_constructor
           
            if  multival_type == str:
                string_list = element.value
                value = [format_string(x) for x in s if len(x) > 0]
        elif value_type == list:
            print(key)
            print(value)
            print(vr)
                

Sequence
AcquisitionMatrix
[0, 256, 128, 0]
US
